### Numpy 加速
Python 是一种相对较慢的编程语言，但是我们可以通过使用Numpy来加速Python的运算。Numpy是一个基于C语言的库，提供了许多高效的运算函数，例如矩阵运算和线性代数运算等。这些运算都是基于C语言实现的，因此速度非常快。

GP的性能瓶颈通常在于模型评估。因此，在这里，我们重点关注如何加速评估函数。

In [1]:
import time

import numpy as np
from deap import base, creator, tools, gp


# 符号回归
def evalSymbReg(individual, pset):
    # 编译GP树为函数
    func = gp.compile(expr=individual, pset=pset)
    
    # 使用numpy创建一个向量
    x = np.linspace(-10, 10, 100) 
    
    # 评估生成的函数并计算MSE
    mse = np.mean((func(x) - x**2)**2)
    
    return (mse,)

# 创建个体和适应度函数
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin)



同时，我们还可以考虑将一些算子替换为Numpy函数。尽管这并不是非常重要，因为Numpy已经重载了许多运算符。

In [2]:
import random

# 定义函数集合和终端集合
pset = gp.PrimitiveSet("MAIN", arity=1)
pset.addPrimitive(np.add, 2)
pset.addPrimitive(np.subtract, 2)
pset.addPrimitive(np.multiply, 2)
pset.addPrimitive(np.negative, 1)
pset.addEphemeralConstant("rand101", lambda: random.randint(-1, 1))
pset.renameArguments(ARG0='x')

# 定义遗传编程操作
toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=2)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)
toolbox.register("evaluate", evalSymbReg, pset=pset)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr, pset=pset)

C:\Users\zhenl\anaconda3\Lib\site-packages\deap\gp.py:254: RuntimeWarning: Ephemeral rand101 function cannot be pickled because its generating function is a lambda function. Use functools.partial instead.
  warnings.warn("Ephemeral {name} function cannot be "


现在，让我们来测试一下加速效果。

In [3]:
import numpy
from deap import algorithms

# 定义统计指标
stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
stats_size = tools.Statistics(len)
mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
mstats.register("avg", numpy.mean)
mstats.register("std", numpy.std)
mstats.register("min", numpy.min)
mstats.register("max", numpy.max)

# 使用默认算法
start=time.time()
population = toolbox.population(n=300)
hof = tools.HallOfFame(1)
pop, log  = algorithms.eaSimple(population=population,
                           toolbox=toolbox, cxpb=0.5, mutpb=0.2, ngen=50, stats=mstats, halloffame=hof, verbose=True)
end=time.time()
print('time:',end-start)
print(str(hof[0]))

   	      	                    fitness                    	                      size                     
   	      	-----------------------------------------------	-----------------------------------------------
gen	nevals	avg    	gen	max   	min	nevals	std    	avg    	gen	max	min	nevals	std    
0  	300   	2678.55	0  	151631	0  	300   	8686.15	4.16333	0  	7  	2  	300   	1.75973
1  	161   	3087.55	1  	153712	0  	161   	12461.1	4.31667	1  	13 	2  	161   	2.07759
2  	178   	3250.26	2  	153712	0  	178   	15065.2	4.79   	2  	13 	2  	178   	2.09902
3  	176   	4531.73	3  	159956	0  	176   	21531.6	4.85   	3  	13 	2  	176   	2.07866
4  	178   	47079.9	4  	1.17257e+07	0  	178   	676813 	4.89   	4  	13 	2  	178   	2.23858
5  	182   	4826.69	5  	162173     	0  	182   	24785.2	4.54667	5  	13 	2  	182   	2.17282
6  	186   	7083.44	6  	153712     	0  	186   	30146.7	4.15667	6  	11 	2  	186   	1.78665
7  	174   	5444.15	7  	159956     	0  	174   	26248.1	4.05333	7  	12 	2  	174   	1.77496
8  	184   

对比下面的慢速评估，使用Numpy的加速效果还是非常明显的。

In [4]:
# 慢速评估
def evalSymbRegSlow(individual, pset):
    # 编译GP树为函数
    func = gp.compile(expr=individual, pset=pset)
    
    # 创建评估数据
    xs = [x/5.0 for x in range(-50, 51)]
    
    # 评估生成的函数并计算MSE
    mse = sum((func(x) - x**2)**2 for x in xs) / len(xs)
    
    return (mse,)

toolbox.register("evaluate", evalSymbRegSlow, pset=pset)

start=time.time()
population = toolbox.population(n=300)
hof = tools.HallOfFame(1)
pop, log  = algorithms.eaSimple(population=population,
                           toolbox=toolbox, cxpb=0.5, mutpb=0.2, ngen=50, stats=mstats, halloffame=hof, verbose=True)
end=time.time()
print('time:',end-start)

   	      	                    fitness                    	                      size                     
   	      	-----------------------------------------------	-----------------------------------------------
gen	nevals	avg    	gen	max   	min	nevals	std    	avg    	gen	max	min	nevals	std    
0  	300   	4688.79	0  	153622	0  	300   	19458.5	3.95333	0  	7  	2  	300   	1.67466
1  	197   	3048.19	1  	153622	0  	197   	12402.9	4.54   	1  	13 	2  	197   	2.11858
2  	168   	42710.3	2  	1.17166e+07	0  	168   	675347 	5.20667	2  	16 	2  	168   	2.44349
3  	191   	43650.6	3  	1.20176e+07	0  	191   	692696 	6.07667	3  	20 	2  	191   	2.80905
4  	171   	8901.24	4  	164059     	0  	171   	33229.1	6.46   	4  	21 	2  	171   	2.84987
5  	184   	88259  	5  	1.1864e+07 	0  	184   	960299 	6.51667	5  	21 	2  	184   	2.79697
6  	169   	7550.51	6  	159864     	0  	169   	31164.7	6.38667	6  	18 	2  	169   	2.8654 
7  	191   	49675.1	7  	1.18723e+07	0  	191   	685351 	6.28   	7  	16 	2  	191   	2.99248
